In [1]:
%%capture
!git clone https://github.com/HackerBschor/SofteningQueryEvaluation
%cd SofteningQueryEvaluation

!pip3 install faiss-gpu-cu12
!pip3 install pgvector
!pip3 install kagglehub
!pip3 install scikit-learn

In [2]:
from huggingface_hub import notebook_login
notebook_login()

# Evaluate Similarity Grouping


In [3]:
import pandas as pd
import numpy as np

from tqdm import tqdm

from models import ModelMgr
from models.embedding.SentenceTransformer import SentenceTransformerEmbeddingModel
from models.semantic_validation import LLaMAValidationModel

from db.operators import Dummy, Select
from db.criteria import SoftEqual
from db.structure import Column, Constant
import kagglehub

from evaluation.util import calculate_metrics, calc_bleu

import time

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [4]:
# Models
# meta-llama/Llama-3.2-3B-Instruct
# meta-llama/Meta-Llama-3-8B-Instruct

stem = SentenceTransformerEmbeddingModel(ModelMgr())
lsv = LLaMAValidationModel(ModelMgr(), model_path = "meta-llama/Meta-Llama-3-8B-Instruct")
# lsv = DeepSeekValidationModel(m)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [5]:
ZERO_SHOT_SYSTEM_PROMPT = "You are a validator. Respond with \"no\" and \"yes\" only!"
ZERO_SHOT_PROMPTING_TEMPLATE = 'Does "{}" describes "{}"'

In [6]:
path = kagglehub.dataset_download("uciml/zoo-animal-classification")

100%|██████████| 1.85k/1.85k [00:00<00:00, 4.28MB/s]

Extracting files...


In [7]:
classes = pd.read_csv(f"{path}/class.csv", index_col=0)[["Class_Type"]]
classes

,Class_Type
Class_Number,
1,Mammal
2,Bird
3,Reptile
4,Fish
5,Amphibian
6,Bug
7,Invertebrate


In [8]:
transforms = {"is": "is not", "has": "has no", "does": "does not", "lays": "does not lays", "gives": "gives no"}
transform_cols = {"hair": "has", "feathers": "has", "eggs": "lays", "milk": "gives", "airborne": "is", "aquatic": "is", "predator": "is", "toothed": "is", "backbone": "has", "breathes": "does", "venomous": "is", "fins": "has", "legs": "has", "tail": "has", "domestic": "is", "catsize": "is" }

columns = ["name"] + [col for col in transform_cols]

print(columns)

zoo = pd.read_csv(f"{path}/zoo.csv", index_col=0)
for col in transform_cols:
    t = transform_cols[col]
    zoo[col] = zoo[col].apply(lambda x: f'{t if x else transforms[t]} {col}')
zoo

['name', 'hair', 'feathers', 'eggs', 'milk', 'airborne', 'aquatic', 'predator', 'toothed', 'backbone', 'breathes', 'venomous', 'fins', 'legs', 'tail', 'domestic', 'catsize']


,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class_type
animal_name,,,,,,,,,,,,,,,,,
aardvark,has hair,has no feathers,does not lays eggs,gives milk,is not airborne,is not aquatic,is predator,is toothed,has backbone,does breathes,is not venomous,has no fins,has legs,has no tail,is not domestic,is catsize,1
antelope,has hair,has no feathers,does not lays eggs,gives milk,is not airborne,is not aquatic,is not predator,is toothed,has backbone,does breathes,is not venomous,has no fins,has legs,has tail,is not domestic,is catsize,1
bass,has no hair,has no feathers,lays eggs,gives no milk,is not airborne,is aquatic,is predator,is toothed,has backbone,does not breathes,is not venomous,has fins,has no legs,has tail,is not domestic,is not catsize,4
bear,has hair,has no feathers,does not lays eggs,gives milk,is not airborne,is not aquatic,is predator,is toothed,has backbone,does breathes,is not venomous,has no fins,has legs,has no tail,is not domestic,is catsize,1
boar,has hair,has no feathers,does not lays eggs,gives milk,is not airborne,is not aquatic,is predator,is toothed,has backbone,does breathes,is not venomous,has no fins,has legs,has tail,is not domestic,is catsize,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wallaby,has hair,has no feathers,does not lays eggs,gives milk,is not airborne,is not aquatic,is not predator,is toothed,has backbone,does breathes,is not venomous,has no fins,has legs,has tail,is not domestic,is catsize,1
wasp,has hair,has no feathers,lays eggs,gives no milk,is airborne,is not aquatic,is not predator,is not toothed,has no backbone,does breathes,is venomous,has no fins,has legs,has no tail,is not domestic,is not catsize,6
wolf,has hair,has no feathers,does not lays eggs,gives milk,is not airborne,is not aquatic,is predator,is toothed,has backbone,does breathes,is not venomous,has no fins,has legs,has tail,is not domestic,is catsize,1


In [9]:
gt = {tuple([key] + [x[col] for col in columns[1:]] + [classes.loc[x["class_type"]].values[0]]) for key, x in zoo.iterrows()}
print(str(gt)[0:500], "...", len(gt))

{('gnat', 'has no hair', 'has no feathers', 'lays eggs', 'gives no milk', 'is airborne', 'is not aquatic', 'is not predator', 'is not toothed', 'has no backbone', 'does breathes', 'is not venomous', 'has no fins', 'has legs', 'has no tail', 'is not domestic', 'is not catsize', 'Bug'), ('kiwi', 'has no hair', 'has feathers', 'lays eggs', 'gives no milk', 'is not airborne', 'is not aquatic', 'is predator', 'is not toothed', 'has backbone', 'does breathes', 'is not venomous', 'has no fins', 'has le ... 101


In [10]:
data = [x[:-1] for x in gt]
print(data[0])

('gnat', 'has no hair', 'has no feathers', 'lays eggs', 'gives no milk', 'is airborne', 'is not aquatic', 'is not predator', 'is not toothed', 'has no backbone', 'does breathes', 'is not venomous', 'has no fins', 'has legs', 'has no tail', 'is not domestic', 'is not catsize')


In [11]:
overall_result = {}

def evaluate(method, threshold, system_prompt=ZERO_SHOT_SYSTEM_PROMPT, prompt_template=ZERO_SHOT_PROMPTING_TEMPLATE):
    pred = []
    runtimes = []
    for animal_type in tqdm(classes.values):
        animal_type = animal_type[0]

        d = Dummy("animals", columns, data)
        s = Select(d, SoftEqual(columns, Constant(animal_type), method=method, em=stem, sv=lsv, threshold=threshold, zfs_system_prompt=system_prompt, zfs_prompt_template = prompt_template))

        tic = time.time()
        result = s.open().fetch_all()
        toc = time.time()

        pred.extend([tuple([x[col] for col in columns] + [animal_type]) for x in result])
        runtimes.append(toc - tic)

    scores = calculate_metrics(gt, set(pred), np.mean(runtimes))

    print(method, threshold, scores["F1 Score"])

    return scores, pred

In [12]:
for thresh in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    res, _ = evaluate("threshold", thresh)
    overall_result[("threshold", thresh)] = res
    if res["Recall"] == 0.0:
        break

100%|██████████| 7/7 [00:10<00:00,  1.46s/it]


threshold 0.1 0.2506203473945409


100%|██████████| 7/7 [00:09<00:00,  1.31s/it]


threshold 0.2 0.2717086834733894


100%|██████████| 7/7 [00:09<00:00,  1.32s/it]


...

threshold 0.4 0.386046511627907


100%|██████████| 7/7 [00:09<00:00,  1.32s/it]


threshold 0.5 0.32653061224489793


100%|██████████| 7/7 [00:09<00:00,  1.32s/it]

threshold 0.6 0


In [13]:
res, pred = evaluate("zero-few-shot", None)
overall_result[("zero-few-shot", None)] = res

100%|██████████| 7/7 [01:07<00:00,  9.58s/it]

zero-few-shot None 0.8207547169811322


In [14]:
for thresh in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    res, _ = evaluate("both", thresh)
    overall_result[("both", thresh)] = res
    if res["Recall"] == 0.0:
        break

100%|██████████| 7/7 [01:11<00:00, 10.17s/it]


both 0.1 0.8207547169811322


100%|██████████| 7/7 [01:02<00:00,  8.98s/it]


both 0.2 0.8229665071770336


100%|██████████| 7/7 [00:58<00:00,  8.32s/it]


...

both 0.4 0.7628865979381444


100%|██████████| 7/7 [00:13<00:00,  1.90s/it]


both 0.5 0.328125


100%|██████████| 7/7 [00:09<00:00,  1.33s/it]

both 0.6 0


In [15]:
for key in tqdm(overall_result):
    scores_bleu = calc_bleu(gt, overall_result[key]["pred"])
    for score_bleu in scores_bleu:
        overall_result[key][score_bleu] = scores_bleu[score_bleu]

100%|██████████| 13/13 [04:08<00:00, 19.15s/it]


In [16]:
df_results = pd.DataFrame([{"method": k[0], "threshold": k[1]} | v for k,v in overall_result.items()])
df_results

,method,threshold,Precision,Recall,F1 Score,tp,fn,fp,runtime,pred,bleu1,bleu2,bleu3,bleu4
0,threshold,0.1,0.143262,1.000000,0.250620,101,0,604,1.461641,"{(gnat, has no hair, has no feathers, lays egg...",1.000000,1.000000,1.000000,1.000000
1,threshold,0.2,0.158238,0.960396,0.271709,97,4,516,1.303450,"{(gnat, has no hair, has no feathers, lays egg...",0.999363,0.999358,0.999353,0.999347
2,threshold,0.3,0.168478,0.920792,0.284839,93,8,459,1.318082,"{(boar, has hair, has no feathers, does not la...",0.998727,0.998717,0.998706,0.998695
3,threshold,0.4,0.252280,0.821782,0.386047,83,18,246,1.299673,"{(boar, has hair, has no feathers, does not la...",0.997065,0.997041,0.997015,0.996990
4,threshold,0.5,0.521739,0.237624,0.326531,24,77,22,1.313231,"{(antelope, has hair, has no feathers, does no...",0.981684,0.975662,0.970362,0.965376
5,threshold,0.6,0.000000,0.000000,0.000000,0,101,0,1.321886,{},-1.000000,-1.000000,-1.000000,-1.000000
6,zero-few-shot,NaN,0.783784,0.861386,0.820755,87,14,24,9.581413,"{(gnat, has no hair, has no feathers, lays egg...",0.997281,0.994758,0.993047,0.991700
7,both,0.1,0.783784,0.861386,0.820755,87,14,24,10.169916,"{(gnat, has no hair, has no feathers, lays egg...",0.997281,0.994758,0.993047,0.991700
8,both,0.2,0.796296,0.851485,0.822967,86,15,22,8.977478,"{(gnat, has no hair, has no feathers, lays egg...",0.997126,0.994602,0.992889,0.991541
9,both,0.3,0.798077,0.821782,0.809756,83,18,21,8.313240,"{(kiwi, has no hair, has feathers, lays eggs, ...",0.996645,0.993800,0.992080,0.990725


In [17]:
df_results.to_pickle("Animals_mpnetBaseV2_LLama8B.pkl")
df_results.drop(columns=["pred"]).to_csv("Animals_mpnetBaseV2_LLama8B.csv")